# **Visualización de datos con plotly**

## El primer paso y el gráfico express

In [ ]:
pip uninstall matplotlib --y

Found existing installation: matplotlib 3.2.2
Uninstalling matplotlib-3.2.2:
  Successfully uninstalled matplotlib-3.2.2


In [ ]:
pip install matplotlib=='3.5.2'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install statsmodels=='0.13.2'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 20.8 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [ ]:
pip install numpy=='1.21.6'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install xlrd=='1.2.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 5.4 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
pip install pandas=='1.3.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install plotly=='5.9.0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 25.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px # Plotly Express

In [ ]:
px.line(x = np.arange(100), y = np.cumsum(np.random.randn(100)), title = 'Línea de ejemplo')

In [ ]:
px.scatter(x = np.cumsum(np.random.randn(100)), y = np.random.exponential(size = 100), title = 'Dispersión de ejemplo')

## Datos de ejemplo

En esta guía utilizaremos un conjunto de datos del Banco Mundial que nos muestra cuántos turistas visitan cada país, y cuánto dinero gastan en sus periodos de vacación. 

Los datasets originales tienen años como columnas, nosotros los transformaremos a filas.

Tomado de: https://towardsdatascience.com/visualization-with-plotly-express-comprehensive-guide-eb5ee4b50b57

In [9]:
import pandas as pd
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
def process_world_bank_dataset(path, value_col):

    # Datos
    data_tourism = pd.read_excel(path, skiprows=3)
    # Regiones
    regions = pd.read_excel(path, sheet_name='Metadata - Countries')
    # Unión a código de país
    fdata_tourism = data_tourism.merge(regions[["Country Code","Region"]], on="Country Code")
    # Filtrado de países
    fdata_tourism = fdata_tourism.loc[~fdata_tourism.Region.isna(),:]
    # Eliminado de filas o columnas totalmente perdidas
    fdata_tourism = fdata_tourism.dropna(how='all', axis = 'rows')
    fdata_tourism = fdata_tourism.dropna(how='all', axis = 'columns')
    # Quitamos columnas no relevantes
    fdata_tourism = fdata_tourism.drop(columns=['Indicator Name','Indicator Code'])
    # Pivotado
    fdata_tourism_long = fdata_tourism.melt(id_vars = ['Country Name', 'Country Code', 'Region'],
                                            var_name = 'Year',
                                            value_name = value_col)

    return fdata_tourism_long

In [ ]:
/content/drive/MyDrive/SEE/arrivals.xls

In [17]:
arrivals = process_world_bank_dataset('/content/drive/MyDrive/SEE/arrivals.xls', 'Receipts')
receipts = process_world_bank_dataset('/content/drive/MyDrive/SEE/receipts.xls', 'Arrivals')
tourism = arrivals.merge(receipts, on = ['Country Name', 'Country Code', 'Region', 'Year'])

In [ ]:
tourism.head()


Country Name Country Code                     Region  Year  Arrivals  \
0        Aruba          ABW  Latin America & Caribbean  1995  912000.0   
1  Afghanistan          AFG                 South Asia  1995       NaN   
2       Angola          AGO         Sub-Saharan Africa  1995    9000.0   
3      Albania          ALB      Europe & Central Asia  1995  304000.0   
4      Andorra          AND      Europe & Central Asia  1995       NaN   

      Receipts  
0  554000000.0  
1          NaN  
2   27000000.0  
3   70000000.0  
4          NaN

In [18]:
tourism.tail()

Country Name Country Code                      Region  Year  \
5637        Kosovo          XKX       Europe & Central Asia  2020   
5638   Yemen, Rep.          YEM  Middle East & North Africa  2020   
5639  South Africa          ZAF          Sub-Saharan Africa  2020   
5640        Zambia          ZMB          Sub-Saharan Africa  2020   
5641      Zimbabwe          ZWE          Sub-Saharan Africa  2020   

          Receipts      Arrivals  
5637           NaN           NaN  
5638           NaN           NaN  
5639  2.716000e+09  3.886600e+06  
5640           NaN  5.020000e+05  
5641  6.600000e+07  6.390000e+05

## Tipos de gráficos

Los gráficos construidos con plotly express siempre estarán construidos dentro de una sola función a la que podemos llamar como **px.chart_type()** donde *chart_type* corresponde al tipo de gráfico que querramos llamar.

In [26]:
# Gráfico de líneas
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador', 'Mexico'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales')

In [30]:
# Gráfico de dispersión
px.scatter(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador','Bolivia'])], 
           x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
           marginal_y = 'box', trendline='lowess')

In [ ]:
# Gráfico de barras
px.bar(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
       x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
       barmode = 'relative')

In [32]:
# Histograma
px.histogram(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
             x='Arrivals', title = 'Visitantes internacionales')

In [ ]:
# Treemap
px.treemap(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
           path = ['Country Name', 'Year'], values='Arrivals', title = 'Visitantes internacionales')

## Manejo de parámetros

Parámetros: https://plotly.com/python-api-reference/generated/plotly.express.line.html

In [ ]:
# Colores
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        color_discrete_sequence=['gold', 'blue', 'red', 'black'])

In [ ]:
# Colores
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        color_discrete_map={'Ecuador':'gold', 'Peru':'blue', 'Colombia':'red', 'Chile':'black'})

In [ ]:
# Tooltips
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        hover_name = 'Country Name', 
        hover_data = {'Arrivals':':.1f', 'Region':True, 'Year':False, 'Country Name':False},
        labels = {'Country Name':'País', 'Year':'Año', 'Arrivals':'Visitas', 'Region':'Región'})

In [ ]:
# Facetas
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Spain', 'France', 'Canada'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        facet_col='Region')

In [33]:
# Data Animaciones
new_tourism = tourism.loc[tourism['Country Name'].isin(['Spain', 'France', 'Canada'])]
new_tourism = []
for i in list(range(1995, 2019)):
  tourism_i = tourism.loc[tourism['Country Name'].isin(['Spain', 'France', 'Canada'])]
  tourism_i = tourism_i.loc[pd.to_numeric(tourism_i['Year'])<=i,:]
  tourism_i['Year Up To'] = i
  new_tourism.append(tourism_i)
new_tourism = pd.concat(new_tourism, axis = 0)
new_tourism.Year = pd.to_numeric(new_tourism.Year)

In [34]:
# Animaciones
px.line(data_frame = new_tourism, 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        range_x = [1995, 2018], range_y = [0,230000000], animation_frame = 'Year Up To')

## Actualizar parámetros gráficos

Los gráficos puede ser actualizados usando:
+ fig.update_layout()
+ fig.update_PARAMETRO()
+ Cambiando la figura en forma de diccionario

In [ ]:
# Gráfico de líneas sencillo
grafico = px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
                  x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales')

In [ ]:
grafico.to_dict()

{'data': [{'hovertemplate': 'Country Name=Chile<br>Year=%{x}<br>Arrivals=%{y}<extra></extra>',
   'legendgroup': 'Chile',
   'line': {'color': '#636efa', 'dash': 'solid'},
   'marker': {'symbol': 'circle'},
   'mode': 'lines',
   'name': 'Chile',
   'orientation': 'v',
   'showlegend': True,
   'type': 'scatter',
   'x': array(['1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
          '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
          '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
          '2019', '2020'], dtype=object),
   'xaxis': 'x',
   'y': array([     nan,      nan,      nan,      nan,      nan,      nan,
               nan,      nan,      nan,      nan,      nan,      nan,
               nan, 3713000., 3657000., 3584000., 3963000., 4397000.,
          4457000., 4601000., 5487000., 6712000., 7621000., 6603000.,
          5431000.,      nan]),
   'yaxis': 'y'},
  {'hovertemplate': 'Country Name=Colombia<br>Year=%{x}<br>Arriv

In [ ]:
# Templates (https://plotly.com/python/templates/)
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        template = 'ggplot2')

In [ ]:
# Templates (https://plotly.com/python/templates/)
px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
        x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales',
        template = 'plotly_dark')

In [ ]:
# Actualización parámetro por parámetro (https://plotly.com/python/reference/layout/yaxis/)
grafico = px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
                  x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales')
grafico.update_xaxes(title = 'Eje X - Año')

In [ ]:
# Sliders
grafico = px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
                  x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales')
grafico.update_xaxes(rangeslider_visible = True)

In [ ]:
# Anotaciones
grafico = px.line(data_frame = tourism.loc[tourism['Country Name'].isin(['Colombia', 'Chile', 'Peru', 'Ecuador'])], 
                  x = 'Year', y='Arrivals', color = 'Country Name', title = 'Visitantes internacionales')
grafico.update_layout(annotations=[
                        {"x":5, "y":1000000,
                        "text": "Punto más bajo del turismo",
                        "showarrow":True,
                        "font": {"size": 15}}])

# Ejercicios

## Ejercicio 1
+ Cargue el dataset de gapminder.
+ Cree un scatterplot de gdp_cap vs life_exp, con population como tamaño y continent como color.
+ Coloque el eje x en escala logarítmica utilizando el argumento log_x.
+ Coloque títulos de ejes utilizando el argumento labels.

In [ ]:
gapminder = pd.read_csv('/content/drive/MyDrive/data/gapminder.csv')
gapminder = gapminder.drop(gapminder.columns[0], axis=1)